In [6]:
import torch
import numpy as np
import pickle

from torch import nn


In [2]:
with open(r'D:\dstore\nlp\w2v\train1-1.npy', 'rb') as f:
    trainx = np.load(f)


In [4]:
trainx = [(x[0], *x[1], *x[2]) for x in trainx]

In [7]:
trainx = np.array(trainx, dtype=np.uint32)

In [10]:
with open(r'D:\dstore\nlp\w2v\train1-1.npy', 'wb') as f:
    np.save(f, trainx)

In [2]:
with open(r'D:\dstore\nlp\w2v\debug_data', 'rb') as f:
    trainx = pickle.load(f)

In [3]:


class Embers(nn.Module):

    def __init__(self, vocab_size, emb_complexity):
        torch.manual_seed(1024)
        super(Embers, self).__init__()
        self.focus = nn.Embedding(vocab_size, emb_complexity)
        self.contrast = nn.Embedding(vocab_size, emb_complexity)

    def new(self):
        self.apply(nn.init.xavier_uniform_(self.focus.weight))
        self.apply(nn.init.xavier_uniform_(self.contrast.weight))
        #self.core_vecs = self.core_vecs.to('cuda')
        #self.delta_vecs = self.delta_vecs.to('cuda')


class Embedding_Trainer:
    def __init__(self, dataset):
        self.dataset = dataset
        self.emb = Embers(6000000, 128)
        self.lossfn = nn.BCEWithLogitsLoss(reduction='none')
        self.optimizer = torch.optim.NAdam(self.emb.parameters())

    def tensor_batch(self, bsize):
        while len(self.dataset) >= bsize:
            hold = []
            for x in range(bsize):
                hold.append(self.dataset.pop())
            fpack, cpack = [], []
            for x in hold:
                fpack.append(x[0])
                cpack.append([*x[1], *x[2]])
            yield (
                torch.bmm(self.emb.focus(torch.tensor(fpack).reshape(-1, 1)), self.emb.contrast(torch.tensor(cpack)).permute(0, 2, 1)).type(torch.FloatTensor), 
                torch.tensor([[1, 1, 1, 1, 0, 0, 0, 0] for _ in range(bsize)]).reshape(-1, 1, 8).type(torch.FloatTensor)
            )

    def train(self):
        dgen = self.tensor_batch(1)
        while True:
            tdata, labels = dgen.__next__()
            loss = self.lossfn(tdata, labels)
            loss.sum().backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            break



In [4]:
emtrain = Embedding_Trainer(trainx)

In [3]:
te1 = nn.Embedding(6000000, 128)
te2 = nn.Embedding(6000000, 128)
t1 = te1(torch.tensor(trainx[1][0], dtype=torch.int64))
t2 = te2(torch.tensor(trainx[1][1], dtype=torch.int64))

In [23]:
loss(torch.bmm(sample[0], sample[1].permute(0, 2, 1)).type(torch.FloatTensor), sample[2].type(torch.FloatTensor))[0]

tensor([1.8026e-02, 8.9016e-01, 2.5536e+00, 2.8011e-03, 4.0764e+00, 6.2964e-01,
        1.0513e+00, 1.5590e+00], grad_fn=<SelectBackward0>)

In [28]:
loss2(torch.bmm(sample[0], sample[1].permute(0, 2, 1)).type(torch.FloatTensor), sample[2].type(torch.FloatTensor)).sum()

tensor(6635.1343, grad_fn=<SumBackward0>)

In [5]:
emtrain.train()